<a href="https://colab.research.google.com/github/JaeyeongLee1/DSAIL_summer_internship/blob/main/PMF_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import pandas as pd
import math
from google.colab import drive

#data preprocessing(movielens data)
drive.mount('/content/drive')

ratings = pd.read_csv('/content/drive/MyDrive/DSAIL랩인턴/ratings.csv') #movielens data 사용 https://www.kaggle.com/sengzhaotoo/movielens-small?select=README.txt
movies = pd.read_csv('/content/drive/MyDrive/DSAIL랩인턴/movies.csv')
del ratings['timestamp']
del movies['genres']
df = pd.merge(ratings,movies)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
#Probabilistic Matrix Factorization(PMF) model
def g(x):
  return 1/(1+math.exp(-x))
def g_prime(x):
  return math.exp(-x)/((1+math.exp(-x))**2)

def RMSE(U,V):
  RMSE=0
  for i in range(N):
    for j in range(M):
      if not np.isnan(R[i][j]):
        RMSE += (R[i][j]-g(U[i].dot(V[j].T)))**2
  return RMSE

def E(U,V):
  E=0
  for i in range(N):
    E += 0.5*lambda_u*U[i].dot(U[i].T)
  for j in range(M):
    E += 0.5*lambda_v*V[j].dot(V[j].T)
  for i in range(N):
    for j in range(M):
      if not np.isnan(R[i][j]):
        E += 0.5*(R[i][j]-g(U[i].dot(V[j].T)))**2
  return E

def gradE(U,V,U_batch,V_batch):
  gradE_U,gradE_V = np.zeros((N,D)),np.zeros((M,D))
  for i in U_batch:
    for j in V_batch:
      if not np.isnan(R[i][j]):
        gradE_U[i] += (R[i][j]-g(U[i].dot(V[j].T)))*g_prime(U[i].dot(V[j].T))*(-V[j])
    gradE_U[i] += lambda_u*U[i]
  for j in V_batch:
    for i in U_batch:
      if not np.isnan(R[i][j]):
        gradE_V[j] += (R[i][j]-g(U[i].dot(V[j].T)))*g_prime(U[i].dot(V[j].T))*(-U[i])
    gradE_V[j] += lambda_v*V[j]
  gradE = (gradE_U,gradE_V)
  return gradE

def SGD(U,V,batch_size,learning_rate,epoch): #epoch = epoch per mini batch
  U_batches = []
  V_batches = []
  for i in range(N//batch_size-1):
    U_batches.append(range(batch_size*i,batch_size*(i+1)))
  U_batches.append(range(batch_size*(N//batch_size-1),N))
  for i in range(M//batch_size-1):
    V_batches.append(range(batch_size*i,batch_size*(i+1)))
  V_batches.append(range(batch_size*(M//batch_size-1),M))
  for U_batch in U_batches:
    for V_batch in V_batches:
      for i in range(epoch):
        gradient = gradE(U,V,U_batch,V_batch)
        U=U-learning_rate*gradient[0]
        V=V-learning_rate*gradient[1]
  return U,V

def PMF_fit(R,D,lambda_u,lambda_v,learning_rate,batch_size,epoch_per_batch):
  N = R.shape[0]
  M = R.shape[1]
  U = np.random.rand(N,D)
  V = np.random.rand(M,D)
  return SGD(U,V,batch_size,learning_rate,epoch_per_batch)

In [66]:
lambda_u = 0.01
lambda_v = 0.001
learning_rate=0.05
epoch_per_batch=2000
mini_batch_size = 100
D = 30

#N = len(set(df['userId']))
N=100
#M = len(set(df['movieId']))
M=100

users = list(set(df['userId']))[:N]
movies = list(set(df['movieId']))[:M]
R = np.zeros((N,M))
for i in range(N):
  for j in range(M):
    s1=df[df['userId']==users[i]]
    s=s1[s1['movieId']==movies[j]]
    x = s.rating.values
    if len(x)==0:
      R[i][j]=None
    else:
      R[i][j]=x[0]

R=(R-0.5)/4.5

U0 = np.random.rand(N,D)
V0 = np.random.rand(M,D)
print(RMSE(U0,V0)) #RMSE for random U,V
U,V = PMF_fit(R,D,lambda_u,lambda_v,learning_rate,mini_batch_size,epoch)
print(RMSE(U,V)) #RMSE for PMF

87.76306403148718
0.6189239503641449
